In [2]:
import pandas as pd
import numpy as np
from math import exp
from utils import get_names2abbrs_dict

In [3]:
state_po_votes = pd.read_csv('electoral_votes_2020.csv')

In [4]:
d_probs = pd.read_csv('poll_data/all_pres_polls_d_probs.csv')

In [5]:
d_probs

,date,poll_id,question_id,state,state_po,sample_size,population,d_prob
0,2018-11-28,57026,92080,Ohio,OH,648.0,v,0.521739
1,2018-12-17,57170,92351,NaN,NAT,1001.0,rv,0.538462
2,2019-01-07,57275,92597,North Carolina,NC,750.0,rv,0.526882
3,2019-01-21,57365,92773,NaN,NAT,760.0,rv,0.563830
4,2019-01-26,57427,92926,Michigan,MI,600.0,lv,0.569444
...,...,...,...,...,...,...,...,...
5385,2020-11-02,72717,136487,NaN,NAT,914.0,lv,0.536082
5386,2020-11-02,72717,136486,NaN,NAT,1146.0,rv,0.536082
5387,2020-11-02,72717,136485,NaN,NAT,1333.0,a,0.525773
5388,2020-11-02,72714,136475,NaN,NAT,1363.0,lv,0.552083


In [6]:
mask = ((d_probs['date'] >= '2020-05-01') & (d_probs['date'] <= '2020-10-31'))
d_probs = d_probs[mask].sort_values(by='date').reset_index(drop=True)

In [7]:
alpha = 0.4
nat_alpha = 0.0  # Don't include national results at all
standard_sample_size = 1000

In [8]:
def logistic(x):
    k = 0.001
    x0 = 1000
    e = exp(-k * (x - x0))
    return 1/(1+e)

In [9]:
def get_preds(d_probs, alpha, nat_alpha, weight_func):
    all_states = {k: 0.5 for k in set(get_names2abbrs_dict().values())}  # prior = 0.5 for all states
    for row in range(len(d_probs)):
        state_po = d_probs.iloc[row]['state_po']
        sample_size = d_probs.iloc[row]['sample_size'].item()
        d_prob = d_probs.iloc[row]['d_prob'].item()
        sample_weight = weight_func(sample_size)
        if state_po == 'NAT':
            weight = nat_alpha * sample_weight
            for state_code in all_states.keys():
                all_states[state_code] = (all_states[state_code] * (1 - weight)) + (d_prob * weight)
        else:
            weight = alpha * sample_weight
            all_states[state_po] = (all_states[state_po] * (1 - weight)) + (d_prob * weight)
    d_states = list()
    r_states = list()
    for state_po in all_states:
    #     votes = state_po_votes[state_po_votes['state_po'] == state_po]['electoral_votes'].item()
        if all_states[state_po] > 0.5:
            d_states.append(state_po)
        else:
            r_states.append(state_po)
    d_votes = state_po_votes[state_po_votes['state_po'].isin(d_states)]['electoral_votes'].sum()
    r_votes = state_po_votes[state_po_votes['state_po'].isin(r_states)]['electoral_votes'].sum()
    win_str = ''
    win_states = None
    if d_votes >= 270:
        win_str = f'Biden wins with {d_votes} votes!'
        win_states = d_states
    elif r_votes >= 270:
        win_str = f'Trump wins with {r_votes} votes!'
        win_states = r_states
    else:
        win_str = f'It\'s a tie! {d_votes} = {r_votes}'
    print(win_str)
    if win_states:
        print(f'Won with states {sorted(win_states)}')
    return sorted(all_states.items(), key=lambda kv: kv[1])

In [36]:
alpha = 0.4
weight_func = 'logistic'
df = pd.DataFrame.from_dict(get_preds(d_probs, alpha, 0, eval(weight_func))).rename(columns={0: 'state_po', 1: 'd_prob'})
filename = f'predictions/presidential_preds_{"-".join(str(alpha).split("."))}_{weight_func.split(" ", 1)[0]}.csv'
df.to_csv(filename, index=False)
print(f'wrote to {filename}')

Biden wins with 356 votes!
Won with states ['AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'IL', 'MA', 'MD', 'ME', 'MI', 'MN', 'NC', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OR', 'PA', 'RI', 'VA', 'VT', 'WA', 'WI']
wrote to predictions/presidential_preds_0-4_logistic.csv


In [37]:
alpha = 0.2
weight_func = 'lambda x: 1'
df = pd.DataFrame.from_dict(get_preds(d_probs, alpha, 0, eval(weight_func))).rename(columns={0: 'state_po', 1: 'd_prob'})
filename = f'predictions/presidential_preds_{"-".join(str(alpha).split("."))}_{weight_func.split(" ", 1)[0]}.csv'
df.to_csv(filename, index=False)
print(f'wrote to {filename}')

Biden wins with 350 votes!
Won with states ['AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IL', 'MA', 'MD', 'ME', 'MI', 'MN', 'NC', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OR', 'PA', 'RI', 'VA', 'VT', 'WA', 'WI']
wrote to predictions/presidential_preds_0-2_lambda.csv


In [38]:
alpha = 1
weight_func = 'lambda x: 1'
df = pd.DataFrame.from_dict(get_preds(d_probs, alpha, 0, eval(weight_func))).rename(columns={0: 'state_po', 1: 'd_prob'})
filename = f'predictions/presidential_preds_{"-".join(str(alpha).split("."))}_{weight_func.split(" ", 1)[0]}.csv'
df.to_csv(filename, index=False)
print(f'wrote to {filename}')

Biden wins with 388 votes!
Won with states ['AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IL', 'MA', 'MD', 'ME', 'MI', 'MN', 'NC', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OR', 'PA', 'RI', 'TX', 'VA', 'VT', 'WA', 'WI']
wrote to predictions/presidential_preds_1_lambda.csv
